HW2 testing bench  

In [42]:
import numpy as np
import heapq as h
import seaborn as sns
import sys

In [43]:
class Function_node:
    #Globals to be inherited
    node_types = ('+','-','*','/','^','sin','cos','const','var','H')#last is the header indicator (indexing from 1!)
    
    def __init__(self, function_name, children=None, value=None):
             
        #assigning a name in our accepted list
        try:
            if function_name not in self.node_types:
                raise ValueError('"Invalid input. Please enter a valid node type \n\n\t run> heap.node_types for the list"')
            else:
                self.function_name = function_name
        except ValueError as err_msg: 
            #print(err_msg)
            self.function_name = 'ERROR'
            sys.exit() #exit if there is an issue
            
        
        self.children = children if children else []
        self.value = value
        
    def __str__(self): #print statement
        if  self.function_name  == 'const':
            return f"{self.value}"
        elif self.function_name == 'var':
            if self.value: return f"x={self.value}"
            else:return "X"
        
        else:
            return f"{self.function_name}"

In [44]:
# https://github.com/swacad/numpy-heap/blob/master/heap.py
#Starting at index 1~
class NP_Heap(Function_node): 
    def __init__(self,length = 2): 
        self.heap           = np.full(length,np.nan).astype(Function_node)
        self.operators      = ('+','-','*','/','^','sin','cos')
        self.trig_operators = ('sin','cos')

    def get_parent_idx(self, child_idx):
        return int(child_idx/2)

    def get_parent(self, child_idx):
        parent_idx = self.get_parent_idx(child_idx)
        return self.heap[parent_idx] 
    
    def get_left_child_idx(self, parent_idx):
        return 2 * parent_idx

    def get_right_child_idx(self, parent_idx):
        return 2 * parent_idx + 1

    def get_left_child(self, parent_idx):
        left_child = self.heap[self.get_left_child_idx(parent_idx)]
        return left_child

    def get_right_child(self, parent_idx):
        right_child = self.heap[self.get_right_child_idx(parent_idx)]
        return right_child

    def get_children_idx(self, parent_idx):
        return 2 * parent_idx, 2 * parent_idx + 1

    def get_children(self, parent_idx):
        child_1_idx, child_2_idx = self.get_parent_idx(parent_idx)
        return self.heap[child_1_idx], self.heap[child_2_idx]
    
    def print_arr(self):
        heap_str = [str(node) for node in self.heap ]
        print(np.stack((np.arange(self.heap.size),heap_str)))
        return None
    
    def show_function(self): #TODO
        #evaluates a node given its index
        heap_str = [str(node) for node in self.heap ]
        depth = None #some function of length (1/2)
        function_str = None
        return None
    
    # EVALUATE A NODE:
    def node_eval(self,node_ind,X = None): #TODO
        #evaluates a node given its index
        def node_operation(operator,operand):
            if   operator == '+':   return  operand[0] + operand[1]
            elif operator == '-':   return  operand[0] - operand[1]
            elif operator == '*':   return  operand[0] * operand[1]
            elif operator == '/':   return  operand[0] / operand[1]
            elif operator == '^':   return  operand[0] ** operand[1]
            elif operator == 'sin': return  np.sin(operand)#TODO sin/cos try error for 2 val operand (np will do it with no issue)
            elif operator == 'cos': return  np.cos(operand) 
       
        L_child, R_child = self.get_children(node_ind)
        
        #TODO: deal with NAN child if non trig or dual nan for operator leaf

        #CHECKS left child, if an operator, evaluate recursively returning a constant. If X, assign it. 
        if L_child.function_name in self.operators: 
            L_child = self.node_eval(2*node_ind)
        elif L_child.function_name == 'var':
            L_child.value = X
             
        #same for right child
        if R_child.function_name in self.operators: 
            R_child = self.node_eval(2*node_ind+1) 
        elif R_child.function_name == 'var':
            R_child.value = X
        
        # terminating state: both children are constandts (floats) after being evaluated
        node = self.heap[node_ind]
        return node_operation(node,(L_child, R_child))


   
       

    def Mutation(self):  #TODO
        #small change to constants and addition/substraction to variables

        return None

    ##### BLOAT PROBLEM ####
    def pruning(self):   #TODO
        #deletes useless tree
        return None
    
    def constant_consolidation(self): #TODO
        #deletes useless tree
        return None
    
    
    
    def insert(self,parent_indx, node_obj, position = None):
        # check for size availibility or resize
        
        #check heap length if short, doubles array until acceptable
        while self.heap.size -1 < self.get_right_child_idx(parent_indx):
            #print('doubled',self.heap.size ,self.get_right_child_idx(parent_indx)  )
            self.heap = np.append(self.heap, 
                                  np.full(self.heap.size, np.nan).astype(Function_node)) 
                   
        
        #TODO raise error for operators not in the predefined list of self.operators: 
        
        L_indx,R_indx = self.get_children_idx(parent_indx)

        #if a position is provided it will insert with replacemnt
        if position:
            if position   == 'L': self.heap[L_indx] = node_obj
            elif position == 'R': self.heap[R_indx] = node_obj
            else: print("invalid arg position = 'L' or 'R'")
            
        #if no position provided it will insert left to right if a child is empty
        elif np.isnan(self.heap[L_indx]): self.heap[L_indx] = node_obj
        elif np.isnan(self.heap[R_indx]): self.heap[R_indx] = node_obj
            
        else: #insert to the left child. #TODO: implement recursive insert. 
            print('Parent children filled') 
            #self.insert(self,parent_indx = L, value = value )
        return None


In [45]:
hh = NP_Heap(length = 2)
hh.heap[0] = Function_node('H')
hh.heap[1] = Function_node('+')
hh.insert(parent_indx = 1,position= 'L', node_obj = Function_node('*'))
hh.insert(parent_indx = 1,position= 'R', node_obj = Function_node('const', value = 7))
hh.insert(parent_indx = 2,position= 'L', node_obj = Function_node('const', value = 2))
hh.insert(parent_indx = 2,position= 'R', node_obj = Function_node('var', value = 2))

hh.print_arr()
hh.node_types


[['0' '1' '2' '3' '4' '5' '6' '7']
 ['H' '+' '*' '7' '2' 'x=2' 'nan' 'nan']]


('+', '-', '*', '/', '^', 'sin', 'cos', 'const', 'var', 'H')